In [1]:
import socket
import hmac
import numpy as np
import time

class UDPReceiver:
    def __init__(self, X, Y, key, ip='0.0.0.0', port=23422):
        self.X = X
        self.Y = Y
        self.key = key
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.sock.bind((ip, port))
        self.buffer = [b''] * X.shape[0]
        self.macs = [b''] * X.shape[1]
        self.res = {}
        self.total_mac_length = 0
        self.total_data_length = 0
        self.total_verified_data_length = 0
        self.expected_chunk_index = 1
        self.mac_computations = 0  # Tracks the number of MAC computations
        self.buffer_operations = 0  # Tracks the number of buffer operations

    def data_parser(self, data):
        chunk_index = int.from_bytes(data[:4], 'big')
        if np.sum(self.Y[chunk_index % self.X.shape[0] - 1]):
            chunk_data = data[4:-48]
            mac = data[-48:]
        else:
            chunk_data = data[4:]
            mac = b''
        return chunk_index, chunk_data, mac

    def fill_buffer_mac(self, chunk_index, chunk_data, mac):
        self.buffer[chunk_index % self.X.shape[0] - 1] = chunk_data
        try:
            self.macs[np.where(self.Y[chunk_index % self.X.shape[0] - 1] == 1)[0][0]] = mac
        except IndexError:
            pass
        self.buffer_operations += 1

    def reset_buffer_mac(self):
        self.buffer = [b''] * self.X.shape[0]
        self.macs = [b''] * self.X.shape[1]

    def check_mac_for_buffer(self, excepted_chunk_index):
        buffer_verified = [b'*' * 2] * self.X.shape[0]

        for tags in range(self.X.shape[1]):
            data = b''.join(self.buffer[np.where(self.X[:, tags] == 1)])
            mac = hmac.new(self.key, data, 'sha384').digest()
            self.mac_computations += 1  # Count the MAC computation
            if mac == self.macs[tags]:
                for x in np.where(self.X[:, tags] == 1)[0]:
                    buffer_verified[x] = self.buffer[x]

        cnt = (excepted_chunk_index - 1) - (excepted_chunk_index - 1) % self.X.shape[0]
        for x in buffer_verified:
            self.res[cnt] = x
            cnt += 1

        self.reset_buffer_mac()

    def receive_data(self):
        print("****Receiving in 2D mode******\n")
        start = time.time()

        while True:
            data, _ = self.sock.recvfrom(4096)  # 8 bytes for index + 1024 bytes of data
            if data == b'END':
                self.finalize()
                break

            chunk_index, chunk_data, mac = self.data_parser(data)
            self.total_mac_length += len(mac)

            if self.expected_chunk_index == chunk_index:
                self.fill_buffer_mac(chunk_index, chunk_data, mac)
                self.expected_chunk_index += 1
            else:
                left_to_verify = self.X.shape[0] - self.expected_chunk_index % self.X.shape[0]
                if self.expected_chunk_index % self.X.shape[0] == 0 or (
                        chunk_index - self.expected_chunk_index) > left_to_verify:
                    self.check_mac_for_buffer(self.expected_chunk_index)

                self.fill_buffer_mac(chunk_index, chunk_data, mac)
                self.expected_chunk_index = chunk_index + 1

            if chunk_index % self.X.shape[0] == 0:
                self.check_mac_for_buffer(self.expected_chunk_index)

        print("Time: ", time.time() - start)

    def finalize(self):
        print("End of transmission\n")

        received_data = b''.join([self.res[x] for x in self.res])
        print("Received data: \n", received_data.decode())

        print(f"\nStats --------------\n\nTotal tag Length: {self.total_mac_length * 8}")
        for x in self.res:
            self.total_data_length += len(self.res[x])
            if b'*' not in self.res[x]:
                self.total_verified_data_length += len(self.res[x])
        print(f"Total Data Length: {self.total_data_length * 8}")
        print(f"Total Verified Data Length: {self.total_verified_data_length * 8}")
        print(f"Goodput: {np.round(self.total_verified_data_length / (self.total_data_length + self.total_mac_length), 2)}")

        # Complexity Metrics
        print(f"MAC Computations: {self.mac_computations}")
        print(f"Buffer Operations: {self.buffer_operations}")
        print(f"Complexity (X): {self.mac_computations / (self.total_data_length + self.total_mac_length):.4f}")
        print(f"Complexity (Y): {self.buffer_operations / (self.total_data_length + self.total_mac_length):.4f}")
        self.sock.close()


# Example usage:
X = np.array([
    [1, 0, 0, 0, 0, 0, 1, 0, 0],  # m1
    [1, 0, 0, 0, 0, 0, 0, 1, 0],  # m2
    [1, 0, 0, 0, 0, 0, 0, 0, 1],  # m3
    [0, 1, 0, 0, 0, 0, 1, 0, 0],  # m4
    [0, 1, 0, 0, 0, 0, 0, 1, 0],  # m5
    [0, 1, 0, 0, 0, 0, 0, 0, 1],  # m6
    [0, 0, 1, 0, 0, 0, 1, 0, 0],  # m7
    [0, 0, 1, 0, 0, 0, 0, 1, 0],  # m8
    [0, 0, 1, 0, 0, 0, 0, 0, 1],  # m9
    [0, 0, 0, 1, 0, 0, 1, 0, 0],  # m10
    [0, 0, 0, 1, 0, 0, 0, 1, 0],  # m11
    [0, 0, 0, 1, 0, 0, 0, 0, 1],  # m12
    [0, 0, 0, 0, 1, 0, 1, 0, 0],  # m13
    [0, 0, 0, 0, 1, 0, 0, 1, 0],  # m14
    [0, 0, 0, 0, 1, 0, 0, 0, 1],  # m15
    [0, 0, 0, 0, 0, 1, 1, 0, 0],  # m16
    [0, 0, 0, 0, 0, 1, 0, 1, 0],  # m17
    [0, 0, 0, 0, 0, 1, 0, 0, 1]   # m18
])

Y = np.array([
    [0, 0, 0, 0, 0, 0, 1, 0, 0],  # m1
    [0, 0, 0, 0, 0, 0, 0, 1, 0],  # m2
    [1, 0, 0, 0, 0, 0, 0, 0, 0],  # m3
    [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m4
    [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m5
    [0, 1, 0, 0, 0, 0, 0, 0, 0],  # m6
    [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m7
    [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m8
    [0, 0, 1, 0, 0, 0, 0, 0, 0],  # m9
    [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m10
    [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m11
    [0, 0, 0, 1, 0, 0, 0, 0, 0],  # m12
    [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m13
    [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m14
    [0, 0, 0, 0, 1, 0, 0, 0, 0],  # m15
    [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m16
    [0, 0, 0, 0, 0, 1, 0, 0, 0],  # m17
    [0, 0, 0, 0, 0, 0, 0, 0, 1]   # m18
])

receiver = UDPReceiver(X, Y, key=b'key')
receiver.receive_data()


****Receiving in 2D mode******

